In [77]:
import pandas as pd
import numpy as np

file_path = 'DS_CAP/covid_19_set_2.csv'
df = pd.read_csv(file_path)

print(len(df))
df.columns 

49068


Index(['ID', 'Province/State', 'Country/Region', 'Lat', 'Long', 'Date',
       'Confirmed', 'Deaths', 'Recovered', 'Active', 'WHO Region'],
      dtype='object')

In [78]:
df.describe

<bound method NDFrame.describe of           ID Province/State         Country/Region        Lat       Long  \
0          1            NaN            Afghanistan   33.93911  67.709953   
1          2            NaN                Albania    41.1533    20.1683   
2          3            NaN                Algeria    28.0339     1.6596   
3          4            NaN                Andorra    42.5063     1.5218   
4          5            NaN                 Angola   -11.2027    17.8739   
...      ...            ...                    ...        ...        ...   
49063  49064            NaN  Sao Tome and Principe     0.1864     6.6131   
49064  49065            NaN                  Yemen  15.552727  48.516388   
49065  49066            NaN                Comoros   -11.6455    43.3333   
49066  49067            NaN             Tajikistan     38.861    71.2761   
49067  49068            NaN                Lesotho     -29.61    28.2336   

             Date Confirmed Deaths Recovered Active  

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49068 entries, 0 to 49067
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              49068 non-null  int64 
 1   Province/State  14356 non-null  object
 2   Country/Region  48068 non-null  object
 3   Lat             48068 non-null  object
 4   Long            48068 non-null  object
 5   Date            48068 non-null  object
 6   Confirmed       48068 non-null  object
 7   Deaths          48068 non-null  object
 8   Recovered       48068 non-null  object
 9   Active          48068 non-null  object
 10  WHO Region      48068 non-null  object
dtypes: int64(1), object(10)
memory usage: 4.1+ MB


In [80]:
df['WHO Region'].unique()

array(['Eastern Mditerranean', 'Europe', 'Africa', nan, 'Americas',
       'Western Pacific', 'Eastern Mediterranean', 'South-East Asia',
       'Arica', 'Europe ', 'E urope', 'Western Paicfic', 'Euope',
       'Western P acific', 'Americs', 'Ameircas', 'Euroep', 'Europ',
       'Am ericas', 'Amer icas', 'Afric', 'Wester Pacific', 'Afria',
       'urope', 'Afica', 'Ameri cas', 'Eurpoe', 'Euroe', 'Eu rope',
       'Eastern Me diterranean', 'Afr ica', 'Americas ', 'Eur ope',
       'Easetrn Mediterranean', 'Wstern Pacific', 'Aericas',
       'aEstern Mediterranean', 'Western Paciifc', 'Western Paific',
       'Ameriacs', 'Afirca', 'Americ as', 'Amricas', 'Europ e',
       'Eastern Mediterr anean', 'frica', 'Western Pa cific',
       'South-East sia', 'Afriac', 'Western Pac ific', 'Afri ca',
       'Euorpe', 'Wsetern Pacific', 'estern Pacific', 'Westren Pacific',
       'Western Pacifc', 'Westen Pacific', 'Westenr Pacific',
       'We stern Pacific', 'West ern Pacific', 'Eurpe', 'fArica',

In [81]:
df.head()

,ID,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,1,NaN,Afghanistan,33.93911,67.709953,2020/01/22,0,0,0,0,Eastern Mditerranean
1,2,NaN,Albania,41.1533,20.1683,2020/01/22,0,0,0,0,Europe
2,3,NaN,Algeria,28.0339,1.6596,2020/01/22,0,0,0,0,Africa
3,4,NaN,Andorra,42.5063,1.5218,2020/01/22,0,0,0,0,Europe
4,5,NaN,Angola,-11.2027,17.8739,2020/01/22,0,0,0,0,Africa


In [82]:
df.tail()

,ID,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
49063,49064,NaN,Sao Tome and Principe,0.1864,6.6131,2020/07/27,865,14,734,117,Africa
49064,49065,NaN,Yemen,15.552727,48.516388,2020/07/27,1691,483,833,375,Eastern Mediterranean
49065,49066,NaN,Comoros,-11.6455,43.3333,2020/07/27,354,7,328,19,Africa
49066,49067,NaN,Tajikistan,38.861,71.2761,2020/07/27,7235,60,6028,1147,Europe
49067,49068,NaN,Lesotho,-29.61,28.2336,2020-07-27,505,12,128,365,Africa


In [83]:
df_copy = df.copy()

In [84]:
import re
initial_rows = len(df)

# Dictionary to track flagged row counts
flagged_counts = {}
rows_to_remove = set()

# Identify rows with all missing values
missing_rows = df[df.isnull().all(axis=1)]
flagged_counts['Missing Rows'] = len(missing_rows)
rows_to_remove. update(missing_rows.index)

# Convert numeric columns properly
num_cols = ['Confirmed', 'Deaths', 'Recovered', 'Active']
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')
negative_values = df[(df[num_cols] < 0).any(axis=1)]
flagged_counts['Negative Values'] = len(negative_values)
rows_to_remove.update(negative_values.index)

incorrect_active = df[df['Active'] != (df['Confirmed'] - df['Deaths'] - df['Recovered'])]
flagged_counts['Incorrect Active Calculation'] = len(incorrect_active)
rows_to_remove.update(incorrect_active.index)

# Identify rows with incorrect WHO Region names (excluding missing values)
valid_regions = {"Africa", "Americas", "Eastern Mediterranean", "Europe", "South-East Asia", "Western Pacific"}
# Strip spaces and convert to title case
df['WHO Region'] = df['WHO Region'].str.strip().str.title()
who_region_issues = df[~df['WHO Region'].isin(valid_regions) ]
flagged_counts['WHO Region Issues'] = len(who_region_issues)
rows_to_remove.update(who_region_issues.index)

df['Lat'] = pd.to_numeric(df['Lat'], errors='coerce')
df['Long'] = pd.to_numeric(df['Long'], errors='coerce')
# Identify rows with Lat/Long outside valid ranges
invalid_latlong = df[(df['Lat'] < -90) | (df['Lat'] > 90) | (df['Long'] < -180) | (df['Long'] > 180)]
non_numeric_latlong = df[df[['Lat', 'Long']].isna().any(axis=1)]
flagged_counts['Invalid Lat/Long'] = len(invalid_latlong)
flagged_counts['Non-Numeric Lat/Long'] = len(non_numeric_latlong)
rows_to_remove.update(invalid_latlong.index)
rows_to_remove.update(non_numeric_latlong.index)

# Identify duplicates
duplicates = df[df.duplicated()]

flagged_counts['duplicates'] = len(duplicates)
rows_to_remove.update(duplicates.index)

# Convert 'Date' to datetime format, forcing errors to NaT (invalid dates) assuming it is in year/month/day
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d', errors='coerce')

invalid_dates = df[df['Date'].isna()]
flagged_counts['Invalid Date Format'] = len(invalid_dates)
rows_to_remove.update(invalid_dates.index)
#origin of covid 19 is in 2019, which means the dates can't be older than that and are "bad"
invalid_dates_old = df[df['Date'] < pd.Timestamp('2019-01-01')]
flagged_counts['Dates Older Than 2019'] = len(invalid_dates_old)
rows_to_remove.update(invalid_dates_old.index)

# Identify rows with special characters in textual columns
def contains_special_chars(value):
    if pd.isna(value): 
        return False
    return bool(re.search(r'[^a-zA-Z0-9\s.,\-\'/]', str(value))) 

text_cols = ['WHO Region', 'Country/Region', 'Province/State']
# Identify rows with special characters in text columns
special_char_mask = pd.DataFrame({col: df[col].map(contains_special_chars) for col in text_cols})
corrupt_data = df[special_char_mask.any(axis=1)]
flagged_counts['Special Characters in Text Columns'] = len(corrupt_data)
rows_to_remove.update(corrupt_data.index)

total_flagged = len(rows_to_remove)

print("Flagged Rows Breakdown:")
for issue, count in flagged_counts.items():
    print(f"{issue}: {count}")
    
print(f"\nTotal Unique Rows Flagged for Removal: {total_flagged}")
df_cleaned = df.drop(index=rows_to_remove)
final_rows = len(df_cleaned)
rows_removed = initial_rows - final_rows

print(f"\nInitial row count: {initial_rows}")
print(f"Final row count: {final_rows}")
print(f"Total rows removed: {rows_removed}")

output_file_path = 'DS_CAP/covid19new.csv'
df_cleaned.to_csv(output_file_path, index=False)

expected_min = int(0.15 * initial_rows)
expected_max = int(0.25 * initial_rows)
if expected_min <= rows_removed <= expected_max:
    print("\nRows removed are within expected range.")
else:
    print("\n WARNING: Rows removed are outside expected range! Double-check the filtering criteria.")


Flagged Rows Breakdown:
Missing Rows: 0
Negative Values: 658
Incorrect Active Calculation: 5565
WHO Region Issues: 2039
Invalid Lat/Long: 0
Non-Numeric Lat/Long: 2341
duplicates: 0
Invalid Date Format: 1816
Dates Older Than 2019: 151
Special Characters in Text Columns: 737

Total Unique Rows Flagged for Removal: 9552

Initial row count: 49068
Final row count: 39516
Total rows removed: 9552

Rows removed are within expected range.


In [88]:
removal_log = pd.DataFrame(columns=["ID", "Column Name", "Justification"])
def log_removals(flagged_df, column_name, reason):
    """Logs removed rows with their ID, column name, and justification"""
    temp_log = pd.DataFrame({
        "ID": flagged_df["ID"],
        "Column Name": column_name,
        "Justification": reason
    })
    return temp_log

# Fully missing rows(N.B.: certain rows that are fully blank are being categorized in other areas)
removal_log = pd.concat([removal_log, log_removals(missing_rows, "Entire Row", "Row has all missing values")])

removal_log = pd.concat([removal_log, log_removals(negative_values, "Confirmed/Deaths/Recovered/Active", "Negative values are not valid")])

removal_log = pd.concat([removal_log, log_removals(incorrect_active, "Active", "Active does not match Confirmed - Deaths - Recovered")])

removal_log = pd.concat([removal_log, log_removals(who_region_issues, "WHO Region", "Invalid or misspelled WHO Region")])

removal_log = pd.concat([removal_log, log_removals(invalid_latlong, "Lat/Long", "Lat/Long out of valid range")])

removal_log = pd.concat([removal_log, log_removals(non_numeric_latlong, "Lat/Long", "Lat/Long contains non-numeric values")])

removal_log = pd.concat([removal_log, log_removals(duplicates, "Entire Row", "duplicate rows")])

# Special Characters in Text Columns(WHO Region, Country/Region, Province/State)(N.B.: some misspelling still exist, unsure how 
#to remove without jeopardizing data
removal_log = pd.concat([removal_log, log_removals(corrupt_data, "Text Column(s)", "Special characters detected")])

removal_log = pd.concat([removal_log, log_removals(invalid_dates, "Date", "Invalid or incorrectly formatted date")])
#I want to note that I realized my date removal method is wrongly removing rows with null dates, but I am unsure on how to fix it.
removal_log = pd.concat([removal_log, log_removals(invalid_dates_old, "Date", "Dates are older than origin of covid-19")])

removal_log.to_csv("DS_CAP/removed_rows_log.csv", index=False)


In [86]:
# check for any rows that are still empty (excluding 'ID')
empty_rows = df_cleaned.drop(columns=['ID']).isnull().all(axis=1).sum()
print(f"Number of fully empty rows (excluding ID): {empty_rows}")


Number of fully empty rows (excluding ID): 0


In [87]:
df_cleaned = df.drop(index=rows_to_remove)
print(f"Final row count: {len(df_cleaned)}")

Final row count: 39516
